In [ ]:
import sqlite3
from pathlib import Path
from datetime import datetime
import os

In [ ]:
def create_database_from_sql(sql_file_path, db_name='database.db'):
    """
    Creates a SQLite database from a .sql file
    
    Args:
        sql_file_path (str): Path to the .sql file
        db_name (str): Name of the database file to create
    """
    # Remove existing database file if it exists
    if Path(db_name).exists():
        Path(db_name).unlink()
    
    # Read the SQL file with UTF-8 encoding
    try:
        with open(sql_file_path, 'r', encoding='utf-8') as sql_file:
            sql_script = sql_file.read()
    except UnicodeDecodeError:
        # Try with different encoding if UTF-8 fails
        with open(sql_file_path, 'r', encoding='utf-8-sig') as sql_file:
            sql_script = sql_file.read()
    
    # Connect to the SQLite database (it will be created if it doesn't exist)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    try:
        # Execute the SQL script
        cursor.executescript(sql_script)
        conn.commit()
        print(f"Database '{db_name}' created successfully from '{sql_file_path}'")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    finally:
        conn.close()

An error occurred: near "auto_increment": syntax error
Created database: netflix.db from ./database/Netflix/netflix.sql


In [ ]:
database_paths = ['./database/Chinook/chinook.sql' ]
db_names = ['./database/Chinook/chinook.db']

In [7]:
for i in range(len(database_paths)):
    create_database_from_sql(database_paths[i], db_names[i])
    print(f"Created database: {db_names[i]} from {database_paths[i]}")

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 23902: character maps to <undefined>

## Refining the Netflix database

In [ ]:
def remove_tables_from_db(db_path, tables_to_remove):
    """
    Remove specified tables from a SQLite database
    
    Args:
        db_path (str): Path to the SQLite database file
        tables_to_remove (list): List of table names to remove

    """

    conn = None
    
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Disable foreign key constraints temporarily
        cursor.execute("PRAGMA foreign_keys = OFF")
        
        # Get list of all existing tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
        existing_tables = [table[0] for table in cursor.fetchall()]
        
        # Remove each specified table if it exists
        for table in tables_to_remove:
            if table in existing_tables:
                try:
                    cursor.execute(f"DROP TABLE IF EXISTS {table}")
                    print(f"Successfully removed table: {table}")
                except sqlite3.Error as e:
                    print(f"Error removing table {table}: {e}")
            else:
                print(f"Table not found (skipping): {table}")
        
        # Re-enable foreign key constraints
        cursor.execute("PRAGMA foreign_keys = ON")
        
        # Commit changes
        conn.commit()
        print("\nTable removal complete!")
        
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()



Table not found (skipping): BATCH_JOB_EXECUTION
Table not found (skipping): BATCH_JOB_EXECUTION_CONTEXT
Table not found (skipping): BATCH_JOB_EXECUTION_PARAMS
Table not found (skipping): BATCH_JOB_EXECUTION_SEQ
Table not found (skipping): BATCH_JOB_INSTANCE
Table not found (skipping): BATCH_JOB_SEQ
Table not found (skipping): BATCH_STEP_EXECUTION
Table not found (skipping): BATCH_STEP_EXECUTION_CONTEXT
Table not found (skipping): BATCH_STEP_EXECUTION_SEQ
Table not found (skipping): episode
Error removing table sqlite_sequence: table sqlite_sequence may not be dropped

Table removal complete!


In [ ]:
def convert_and_clean_dates(db_path):
    """
    Convert Unix timestamp columns (with 'date' in name) to readable format
    and remove the original columns after conversion
    """
    conn = None
    try:
        # Verify database exists
        if not os.path.exists(db_path):
            raise FileNotFoundError(f"Database file not found at: {db_path}")

        # Connect to database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Get all tables in the database
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
        tables = [table[0] for table in cursor.fetchall()]

        for table in tables:
            # Get all columns in the table that contain 'date' in their name
            cursor.execute(f"PRAGMA table_info({table})")
            columns = cursor.fetchall()
            date_columns = [col[1] for col in columns if 'date' in col[1].lower()]

            for column in date_columns:
                try:
                    # Check if the column contains Unix timestamps in milliseconds
                    cursor.execute(f"SELECT {column} FROM {table} LIMIT 1")
                    sample_value = cursor.fetchone()
                    
                    if sample_value and sample_value[0] and str(sample_value[0]).isdigit():
                        # Add a new temporary column for the converted date
                        temp_column = f"{column}_temp"
                        cursor.execute(f"ALTER TABLE {table} ADD COLUMN {temp_column} TEXT")

                        # Convert all timestamps
                        cursor.execute(f"SELECT rowid, {column} FROM {table}")
                        rows = cursor.fetchall()

                        for row in rows:
                            rowid, unix_timestamp = row
                            if unix_timestamp is None:
                                continue
                            try:
                                # Convert from milliseconds to seconds
                                timestamp_seconds = int(unix_timestamp) / 1000
                                readable_date = datetime.fromtimestamp(timestamp_seconds).strftime('%Y-%m-%d %H:%M:%S')
                                cursor.execute(f"""
                                    UPDATE {table} 
                                    SET {temp_column} = ? 
                                    WHERE rowid = ?
                                """, (readable_date, rowid))
                            except (ValueError, TypeError):
                                continue

                        # Drop the original column
                        cursor.execute(f"ALTER TABLE {table} DROP COLUMN {column}")
                        # Rename temp column to original name
                        cursor.execute(f"ALTER TABLE {table} RENAME COLUMN {temp_column} TO {column}")

                        print(f"Converted and cleaned {table}.{column}")

                    else:
                        print(f"Skipping {table}.{column} - not Unix timestamps")

                except sqlite3.Error as e:
                    print(f"Error processing {table}.{column}: {e}")
                    conn.rollback()
                    continue

        conn.commit()
        print("\nDate conversion and cleanup complete!")

    except Exception as e:
        print(f"Error: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

Converted and cleaned movie.release_date_old
Converted and cleaned movie.created_date_old
Converted and cleaned movie.modified_date_old
Skipping movie.release_date - not Unix timestamps
Skipping movie.created_date - not Unix timestamps
Skipping movie.modified_date - not Unix timestamps
Converted and cleaned season.release_date_old
Converted and cleaned season.created_date_old
Converted and cleaned season.modified_date_old
Skipping season.release_date - not Unix timestamps
Skipping season.created_date - not Unix timestamps
Skipping season.modified_date - not Unix timestamps
Skipping tv_show.release_date - not Unix timestamps
Converted and cleaned tv_show.created_date_old
Converted and cleaned tv_show.modified_date_old
Skipping tv_show.created_date - not Unix timestamps
Skipping tv_show.modified_date - not Unix timestamps
Converted and cleaned view_summary.end_date_old
Converted and cleaned view_summary.start_date_old
Converted and cleaned view_summary.created_date_old
Converted and clea

In [ ]:
def remove_old_columns(db_path):
    """
    Remove all columns with '_old' in their name from all tables in the database
    """
    conn = None
    try:
        # Verify database exists
        if not os.path.exists(db_path):
            raise FileNotFoundError(f"Database file not found at: {db_path}")

        # Connect to database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Get all tables in the database
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
        tables = [table[0] for table in cursor.fetchall()]

        for table in tables:
            # Get all columns in the table that contain '_old' in their name
            cursor.execute(f"PRAGMA table_info({table})")
            columns = cursor.fetchall()
            old_columns = [col[1] for col in columns if '_old' in col[1].lower()]

            for column in old_columns:
                try:
                    # SQLite doesn't directly support DROP COLUMN in older versions
                    # So we need to recreate the table without the column
                    
                    # 1. Get all columns except the one we're removing
                    cursor.execute(f"PRAGMA table_info({table})")
                    columns_to_keep = [col[1] for col in cursor.fetchall() 
                                     if '_old' not in col[1].lower()]
                    
                    if not columns_to_keep:
                        print(f"⚠ Warning: Table {table} would be empty after removal - skipping")
                        continue
                        
                    # 2. Create new table without the column
                    columns_str = ', '.join(columns_to_keep)
                    cursor.execute(f"""
                        CREATE TABLE {table}_new AS 
                        SELECT {columns_str} FROM {table}
                    """)
                    
                    # 3. Drop original table
                    cursor.execute(f"DROP TABLE {table}")
                    
                    # 4. Rename new table to original name
                    cursor.execute(f"ALTER TABLE {table}_new RENAME TO {table}")
                    
                    print(f"✓ Removed column: {table}.{column}")
                    
                except sqlite3.Error as e:
                    print(f"✗ Error removing {table}.{column}: {e}")
                    conn.rollback()
                    continue

        conn.commit()
        print("\n'_old' column removal complete!")

    except Exception as e:
        print(f"Error: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()



'_old' column removal complete!


In [ ]:
# Path to your database
netflix_db_path = './database/Netflix/netflix.db'  # Update this to your actual path
# Tables to remove (from your image)
tables_to_remove = [
    'BATCH_JOB_EXECUTION',
    'BATCH_JOB_EXECUTION_CONTEXT',
    'BATCH_JOB_EXECUTION_PARAMS',
    'BATCH_JOB_EXECUTION_SEQ',
    'BATCH_JOB_INSTANCE',
    'BATCH_JOB_SEQ',
    'BATCH_STEP_EXECUTION',
    'BATCH_STEP_EXECUTION_CONTEXT',
    'BATCH_STEP_EXECUTION_SEQ',
    'episode',
    'sqlite_sequence'
]

remove_old_columns(netflix_db_path)
convert_and_clean_dates(netflix_db_path)
remove_tables_from_db(netflix_db_path, tables_to_remove)
